# Pre-Photometry before measuring with GALFITM

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from astropy.io import fits

In [28]:
img_name = 'nircam8'
img_dir = f'./io/image/{img_name}/'
img_merged_path = img_dir + 'merged.fits'

match_dir = f'./io/match/{img_name}/'
prep_dir = f'./io/prep/{img_name}/'

## Create Source Catalog

In [29]:
from photutils.segmentation import SegmentationImage
from photutils.segmentation import SourceCatalog
from astropy.wcs import WCS
from astropy.table import Table, hstack

In [30]:
with fits.open(img_merged_path) as hdul:
    segm = SegmentationImage(hdul['SEGMAP'].data)
    w = WCS(hdul['MERGED'].header)
    image_merged_conv = hdul['SCI_BKSUB_CONV'].data

In [31]:
tab_ref_matched_selected = Table.read(match_dir + 'matchlog.csv', format='ascii.csv')
tab_sorted = tab_ref_matched_selected[tab_ref_matched_selected.argsort('label')] # sort by label
tab_sorted = tab_sorted[tab_sorted['label'] != -1]

label_column = np.int64(tab_sorted['label'])
id_column = np.array(tab_sorted['ID'])
ra_column = tab_sorted['RA']
dec_column = tab_sorted['Dec']
comment_column = tab_sorted['comment']
segm_matched = segm.copy()
segm_matched.keep_labels(label_column)

# Do photometry in every band with SourceCatalog

In [32]:
import glob
img_path_list = glob.glob(img_dir + 'hlsp*.fits')
img_path_list.sort()
img_path_list

['./io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f115w_dr0.6_i2d.fits',
 './io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f150w_dr0.6_i2d.fits',
 './io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f200w_dr0.6_i2d.fits',
 './io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f277w_dr0.6_i2d.fits',
 './io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f356w_dr0.6_i2d.fits',
 './io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f410m_dr0.6_i2d.fits',
 './io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f444w_dr0.6_i2d.fits']

In [33]:
PIXAR_SR = 2.11539874851881E-14
ZERO_POINT =  -6.10 -2.5 * np.log10(PIXAR_SR)
print(f'ZERO_POINT: {ZERO_POINT}')
def flux_to_mag(flux, zero_point=ZERO_POINT):
    return -2.5 * np.log10(flux) + zero_point

ZERO_POINT: 28.086519392283982


In [34]:
from utils.zeropoint import calc_zpt

In [35]:
tab_phot = Table([id_column, ra_column, dec_column, comment_column], names=['ID', 'RA', 'Dec', 'comment'])
band_list = []
for i,img_path in enumerate(img_path_list):
    sci = fits.getdata(img_path, hdu='SCI_BKSUB')
    err = fits.getdata(img_path, hdu='ERR')
    hdr_pri = fits.getheader(img_path, extname='PRIMARY')
    band = hdr_pri['FILTER'].split(' ')[0]
    key_tab = f'KronPhot_{band}_mag'
    cat_sci = SourceCatalog(sci, segm_matched, error=err)
    flux_list = cat_sci.kron_flux
    fluxerr_list = cat_sci.kron_fluxerr
    x, y = cat_sci.xcentroid, cat_sci.ycentroid
    mag_list = []
    for j in range(len(flux_list)):
        x_pix, y_pix, flux = x[j], y[j], flux_list[j]
        zpt = calc_zpt(x_pix, y_pix, 'CR0')[i]
        mag = flux_to_mag(flux, zero_point= zpt)
        mag_list.append(mag)
    tab_phot.add_columns([mag_list, flux_list, fluxerr_list], names=[key_tab, key_tab.replace('mag', 'flux'), key_tab.replace('mag', 'fluxerr')])

In [36]:
tab_phot.write(prep_dir + 'kronphot.csv', format='ascii.csv', overwrite=True)
tab_phot

ID,RA,Dec,comment,KronPhot_F115W_mag,KronPhot_F115W_flux,KronPhot_F115W_fluxerr,KronPhot_F150W_mag,KronPhot_F150W_flux,KronPhot_F150W_fluxerr,KronPhot_F200W_mag,KronPhot_F200W_flux,KronPhot_F200W_fluxerr,KronPhot_F277W_mag,KronPhot_F277W_flux,KronPhot_F277W_fluxerr,KronPhot_F356W_mag,KronPhot_F356W_flux,KronPhot_F356W_fluxerr,KronPhot_F410M_mag,KronPhot_F410M_flux,KronPhot_F410M_fluxerr,KronPhot_F444W_mag,KronPhot_F444W_flux,KronPhot_F444W_fluxerr
str10,float64,float64,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
nircam8_8,214.99584044946604,52.85424080725352,--,23.042603385669402,104.12773044017,4.372603957228912,22.913139394764727,117.31458240050677,5.116393964531273,22.771386008455288,133.67597287906628,5.964119068737193,22.69949237380848,142.82716220242446,6.204088891312089,22.68236439131695,145.09819181746838,6.172836107875069,22.84066245538008,125.41306296618743,5.231992653239932,22.975667303644126,110.74926061272032,4.573725645761873
nircam8_12,215.01340455726907,52.867824024287984,--,22.61470151145074,154.4283930652286,5.0446503846057995,22.448591465685865,179.95800527878782,5.969084549145044,22.3329424852714,200.1846452712158,7.023769256517632,22.210436722167415,224.0954651039493,7.664377028624716,22.19130102214466,228.0800754959901,7.663872081397185,22.340196436706968,198.85164234129445,6.601961912520142,22.477357873910986,175.25265552131384,5.764570331466458
nircam8_55,215.00895017002145,52.86750692595524,--,23.647396784383115,59.655301249108504,3.6650680000497227,23.475800506422683,69.86948696490369,4.364889857121366,23.442315976647098,72.05786101359223,4.414835358702207,23.30058774976019,82.10557691459235,4.642720806962777,23.22013490636165,88.42066856620153,4.82157321961389,23.185517458162323,91.2852845478319,4.905039786875029,23.260212918331266,85.21628349928875,4.4872965753402845
nircam8_53,215.0643292955432,52.9079098803777,--,23.22419758298983,88.09042873917097,5.11356774634736,23.074350582254674,101.12709319171873,5.9527874798498255,22.944132113565082,114.01314128174019,6.605265051325229,23.044002159125075,103.993667191186,5.655341663486415,23.55586229717184,64.90271104378951,3.4024821638054052,23.697170652018595,56.98223718385411,2.8641349260402142,23.685127374994202,57.61781793258672,2.8626313083883987
nircam8_47,215.0026358462628,52.866719820049184,--,23.742774180657015,54.63841401834031,1.6121001711664167,23.60850588488822,61.83087649417721,1.8424232414920207,23.569204745603876,64.11001207348765,1.9230310461415572,23.619574598376712,61.20373380061916,1.8085141248737646,24.011117522169933,42.67374404856689,1.2419675212584071,23.950255966466752,45.13416064318545,1.1767424465086773,24.22863892818809,34.926268280014476,0.9881295618419116
nircam8_43,214.98252509927488,52.85438085491478,--,22.554784619724,163.1901377543715,7.743153720782881,22.432669226762968,182.61652157357327,8.649563708942974,22.349879245561972,197.0861286150987,9.436714893645409,22.564587185115414,161.7234073351862,7.265668361942877,22.96930272226992,111.40037890600149,4.7429532389849225,23.197318726333343,90.29844392830684,3.885933026978925,23.208925178255612,89.33830030786687,3.679651746486993
nircam8_42,214.9931858563449,52.86237721492944,--,23.433178347423663,72.66686394851268,5.19362724960707,23.21465460684853,88.86810391302589,6.529353827073422,23.189413060193733,90.95834146765603,6.408898610197964,23.1895602701233,90.94600968459359,5.51391451825258,23.21893026510814,88.51882712123583,5.1780125428564485,23.217154703637227,88.66370504182831,5.16253344325598,23.17588967148735,92.09835820933418,5.239758548319284
nircam8_40,215.03496565736603,52.89408561073651,--,21.75810269109533,339.91214370952497,12.412047429887693,21.319758514479354,508.98425864716364,19.58467329731211,20.853785175274115,781.7964369632948,34.73920101970206,20.409064786523807,1177.556739268267,56.058150535149956,20.45390767270406,1129.9121738

# Write to FITS file for GALFIT preparation

#### IMGLIST Extension

In [37]:
pregalfit_path = prep_dir + 'pregalfit.fits'

# Create an IMGLIST extension
hdr_img = fits.Header()
hdr_img['EXTNAME'] = 'IMGLIST'

tab_img = Table()
band_list = []
for img_path in img_path_list:
    print(img_path)
    hdr_pri = fits.getheader(img_path, extname='PRIMARY')
    band = hdr_pri['FILTER'].split(' ')[0]
    band_list.append(band)
tab_img.add_columns([band_list,img_path_list], names=['band', 'img_path'])
fits.writeto(pregalfit_path, tab_img.as_array(), header=hdr_img, overwrite=True)

./io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f115w_dr0.6_i2d.fits
./io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f150w_dr0.6_i2d.fits
./io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f200w_dr0.6_i2d.fits
./io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f277w_dr0.6_i2d.fits
./io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f356w_dr0.6_i2d.fits
./io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f410m_dr0.6_i2d.fits
./io/image/nircam8/hlsp_ceers_jwst_nircam_nircam8_f444w_dr0.6_i2d.fits


#### INIPARAM Extension

In [38]:
srccat = SourceCatalog(image_merged_conv, segm_matched, wcs=w, convolved_data=image_merged_conv, progress_bar=True)
srctab = srccat.to_table()
srctab.keep_columns(['label', 'xcentroid', 'ycentroid','orientation','eccentricity','semimajor_sigma','semiminor_sigma'])
srctab['label'] = srctab['label'].astype(str)
kron_rad = srccat.kron_radius*np.sqrt(srccat.semimajor_sigma.value*srccat.semiminor_sigma.value)
srctab.add_column(kron_rad, name='kron_radius')
tab_final = hstack([tab_phot,srctab])
tab_final = tab_final[tab_final.argsort('ID')]

kron_radius:   0%|          | 0/62 [00:00<?, ?it/s]

kron_photometry:   0%|          | 0/62 [00:00<?, ?it/s]

In [39]:
# Create a INIPARAM extension
hdr_ini = fits.Header()
hdr_ini['EXTNAME'] = 'INIPARAM'
fits.append(pregalfit_path, tab_final.as_array(), header=hdr_ini, overwrite=True)

hdr_segm = fits.Header()
hdr_segm['EXTNAME'] = 'SEGMAP'
fits.append(pregalfit_path, segm.data, header=hdr_segm, overwrite=True)